In [ ]:
!pip install deepmultilingualpunctuation
import pandas as pd
import csv
import sys
from ast import literal_eval
from deepmultilingualpunctuation import PunctuationModel
import spacy
from spacy.matcher import PhraseMatcher
from transformers import BartTokenizer, BartForConditionalGeneration

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
# Import collected data

csv.field_size_limit(sys.maxsize)
file_path = "../Data collection/youtube_collected_2024.csv"
video_list = []
with open(file_path, newline='') as csvfile:
  csv_reader = csv.DictReader(csvfile)
  for row in csv_reader:
    del row['']
    video_list.append(dict(row))

for video in video_list:
  stocks = literal_eval(video['Stock_indices'])
  video['Stock_indices'] = stocks

pd.DataFrame(video_list)

,Video ID,Title,Channel,Upload date,Transcipt,Stock_indices
0,YEoJq_PcOgc,What can reignite Apple shares?,CNBC Television,2024-02-28 17:55:38+00:00,LATELY. WE ALSO HAVE NEW COMMITTEE MOVES TO DI...,[1]
1,Hp6IRjNJuI4,Jim Cramer on what could be the next $1 trilli...,CNBC Television,2024-02-27 00:23:35+00:00,"""MAD MONEY"" WITH JIM CRAMER STARTS RIGHT NOW....",[]
2,GQK4bCZHudM,Top tech stocks don't look 'wildly expensive' ...,CNBC Television,2024-03-06 14:24:00+00:00,"WE GOT ADT TODAY MIGHT BE INTERESTING, TOO. TH...",[]
3,BOm0zNiaNjg,Alphabet's AI problems: Stock falls 4%,CNBC Television,2024-02-26 19:01:28+00:00,♪ >>> WELCOME BACK. LET'S TALK ALPHABET. IT I...,[3]
4,Z0D_FwSF5z0,Jim Cramer talks the 'New Industrial Revolutio...,CNBC Television,2024-03-07 00:38:42+00:00,"LIKE. >> ""MAD MONEY"" WITH JIM CRAMER STARTS ...",[]
...,...,...,...,...,...,...
1253,m_gn1XIUNFo,US Supreme Court to hear Donald Trump immunity...,BBC News,2024-02-29 09:15:01+00:00,some breaking news because the Supreme Court h...,[]
1254,RXmMUEGLHEc,Donald Trump can stay on presidential ballot r...,BBC News,2024-03-04 15:33:59+00:00,[Music] now we're going to move away from the ...,[]
1255,S6hF2ffyz2c,Farmers set fires in Brussels ahead of Agricul...,BBC News,2024-02-26 11:42:30+00:00,agricultural ministers from across the Europea...,[]
1256,pyGyrz06ljg,Israeli protesters block food convoys for star...,BBC News,2024-03-05 23:46:00+00:00,President Biden has warned Israel that there a...,[]


Step 1: Keep videos identified with S&P 500 stocks for pre-processing

In [ ]:
cleaned_list = []

for video in video_list:
  if video['Stock_indices']:
    video['Transcipt'] = video['Transcipt'].replace('\n', ' ')
    cleaned_list.append(video)

pd.DataFrame(cleaned_list)

,Video ID,Title,Channel,Upload date,Transcipt,Stock_indices
0,YEoJq_PcOgc,What can reignite Apple shares?,CNBC Television,2024-02-28 17:55:38+00:00,LATELY. WE ALSO HAVE NEW COMMITTEE MOVES TO DI...,[1]
1,BOm0zNiaNjg,Alphabet's AI problems: Stock falls 4%,CNBC Television,2024-02-26 19:01:28+00:00,♪ >>> WELCOME BACK. LET'S TALK ALPHABET. IT I...,[3]
2,cTncTPylZQ8,Apple shares touch a 4-month low,CNBC Television,2024-03-05 17:54:20+00:00,AT THAT LINE BUT DID DIP BELOW AS I LOOK AT I...,[1]
3,qAITe2Hn8Hc,"Three-Stock Lunch: Broadcom, Kroger & Costco",CNBC Television,2024-03-07 20:47:58+00:00,">> WOW, CHIPS, CHIPS, CHIPS. MAKES ME HUNGRY ...","[8, 23, 217]"
4,G573i2zmNkk,"Committee stocks on the move: Costco, Eli Lill...",CNBC Television,2024-03-08 18:20:25+00:00,"COSTCO, MIXED QUARTER. JASON SNIPE, REV MISS, ...","[7, 23, 27]"
...,...,...,...,...,...,...
281,2LMafm5E1TY,Meet the Press NOW — Feb. 26,NBC News,2024-02-27 00:15:00+00:00,>>> FORMER FORMER PRESIDENT >>> FORMER FORMER ...,[52]
282,GkWha5al8Sk,Meet the Press NOW — Feb. 27,NBC News,2024-02-27 23:45:00+00:00,"WE WILL HAVE MUCH MORE HEAD ON MEET THE PRESS,...",[52]
283,xOqh-4THhF8,Texas Gov. Greg Abbott on border crisis and Tr...,ABC News,2024-03-06 03:30:01+00:00,the governor of Texas Greg Abbott Republican G...,[37]
284,Hg-SVFSahI0,"CVS, Walgreens to sell abortion pill: What to ...",CBS News,2024-03-02 01:37:00+00:00,">> ALL RIGHT, OMAR, THANK YOU. >>> THIS MON...","[92, 390]"


Step 2: Add punctuations to video transcripts

In [ ]:
model = PunctuationModel()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


In [ ]:
for video in cleaned_list:
  transcript = video['Transcipt']
  try:
    modified_transcript = model.restore_punctuation(transcript)
    video['Transcipt'] = modified_transcript
  except:
    cleaned_list.remove(video)

pd.DataFrame(cleaned_list)

,Video ID,Title,Channel,Upload date,Transcipt,Stock_indices
0,YEoJq_PcOgc,What can reignite Apple shares?,CNBC Television,2024-02-28 17:55:38+00:00,LATELY. WE ALSO HAVE NEW COMMITTEE MOVES TO DI...,[1]
1,BOm0zNiaNjg,Alphabet's AI problems: Stock falls 4%,CNBC Television,2024-02-26 19:01:28+00:00,♪. >>>. WELCOME BACK. LET'S TALK ALPHABET. IT ...,[3]
2,cTncTPylZQ8,Apple shares touch a 4-month low,CNBC Television,2024-03-05 17:54:20+00:00,AT THAT LINE BUT DID DIP BELOW AS I LOOK AT IT...,[1]
3,qAITe2Hn8Hc,"Three-Stock Lunch: Broadcom, Kroger & Costco",CNBC Television,2024-03-07 20:47:58+00:00,">>. WOW, CHIPS, CHIPS, CHIPS. MAKES ME HUNGRY ...","[8, 23, 217]"
4,G573i2zmNkk,"Committee stocks on the move: Costco, Eli Lill...",CNBC Television,2024-03-08 18:20:25+00:00,COSTCO MIXED QUARTER. JASON SNIPE REV. MISS SE...,"[7, 23, 27]"
...,...,...,...,...,...,...
280,2LMafm5E1TY,Meet the Press NOW — Feb. 26,NBC News,2024-02-27 00:15:00+00:00,>>> FORMER FORMER PRESIDENT. >>>. FORMER FORME...,[52]
281,GkWha5al8Sk,Meet the Press NOW — Feb. 27,NBC News,2024-02-27 23:45:00+00:00,WE WILL HAVE MUCH MORE HEAD ON MEET THE PRESS ...,[52]
282,xOqh-4THhF8,Texas Gov. Greg Abbott on border crisis and Tr...,ABC News,2024-03-06 03:30:01+00:00,"the governor of Texas, Greg Abbott, Republican...",[37]
283,Hg-SVFSahI0,"CVS, Walgreens to sell abortion pill: What to ...",CBS News,2024-03-02 01:37:00+00:00,">>. ALL RIGHT, OMAR, THANK YOU. >>>. THIS MONT...","[92, 390]"


Step 3: Topic segmentation based on different stocks identified in video title

In [ ]:
sp500_df = pd.DataFrame(pd.read_excel('../Data collection/sp500.xlsx'))
sp500_df.drop_duplicates(inplace=True)
nlp = spacy.load("en_core_web_sm")

In [ ]:
def identify_stocks_in_sentence(sentence, identified_stocks):
  doc = nlp(sentence)
  matches = []
  for s in identified_stocks:
    c_lower = sp500_df.iloc[s]['Company'].lower()
    matcher = PhraseMatcher(nlp.vocab)
    patterns =[nlp(c_lower)]
    matcher.add("StockMatcher", None, *patterns)
    for match_id, start, end in matcher(doc):
      matches.append(s)

  return list(set(matches))


def combine_segments(topic_segments):
  combined_data = {}
  stock_record = []
  nested = []

  for seg in topic_segments:
    stocks = seg['stocks']
    sentences = ' '.join(seg['sentences'])

    if stocks:
      for s in stocks:
        if s not in stock_record:
          stock_record.append(s)
          combined_data[s] = {'stock': s, 'sentences': sentences}
        else:
          length = len(nlp(sentences)) + len(nlp(combined_data[s]['sentences']))
          if length < 800:
            combined_data[s]['sentences'] += ' ' + sentences
          else:
            nested.append(combined_data[s])
            combined_data[s] = {'stock': s, 'sentences': sentences}

  for seg in list(combined_data.values()):
    nested.append(seg)

  return nested


def transcript_topic_segmentation(video):
    transcript = video['Transcipt']
    doc = nlp(transcript.lower())

    topic_segments = []
    current_segment = {'stocks': [], 'sentences': []}

    # Iterate over the sentences in the transcript
    for sent in doc.sents:
      # Extract stocks from each sentence
      stocks = identify_stocks_in_sentence(str(sent), video['Stock_indices'])

      # If there are stocks, consider it as a topic segment
      if stocks:
        # Save the current segment if not empty
        if current_segment['sentences']:
          topic_segments.append(current_segment)
          current_segment = {'stocks': [], 'sentences': []}
        # Start a new segment
        current_segment['stocks'] = stocks
        current_segment['sentences'].append(sent.text)
      else:
        # Append the sentence to the current segment
        current_segment['sentences'].append(sent.text)

    # Add the last segment to the list
    if current_segment['sentences']:
        topic_segments.append(current_segment)

    final_segments = combine_segments(topic_segments)
    return final_segments

Step 4: Text summarization on each video segment

In [ ]:
# Load the summarizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def bart_summarize(segment_to_summarize, tokenizer, model):
  # Tokenize the input text
  inputs = tokenizer.batch_encode_plus([segment_to_summarize], return_tensors='pt')

  try:
    # Generate the summary
    summary_ids = model.generate(inputs['input_ids'], max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  except:
    summary = ''
  return summary

In [ ]:
summarized_video_list = []

for video in cleaned_list:
  transcript = video['Transcipt']
  segments = transcript_topic_segmentation(video)
  summarized_segments = {}
  stock_record = []

  for seg in segments:
    current_summary = {'stock': seg['stock'], 'summary': bart_summarize(seg['sentences'], tokenizer, model)}
    if seg['stock'] not in stock_record:
      stock_record.append(seg['stock'])
      summarized_segments[seg['stock']] = current_summary
    else:
      summarized_segments[seg['stock']]['summary'] += ' ' + current_summary['summary']

  for seg in list(summarized_segments.values()):
    video_segment = {}
    video_segment['Video ID'] = video['Video ID']
    video_segment['Title'] = video['Title']
    video_segment['Channel'] = video['Channel']
    video_segment['Upload date'] = video['Upload date']
    video_segment['Stock'] = seg['stock']
    video_segment['Summary'] = seg['summary']
    summarized_video_list.append(video_segment)

Step 5: Save

In [ ]:
for video in summarized_video_list:
  stock_index = video['Stock']
  company = sp500_df.iloc[stock_index]['Company']
  symbol = sp500_df.iloc[stock_index]['Symbol']
  video['Company'] = company
  video['Symbol'] = symbol

pd.DataFrame(summarized_video_list)

,Video ID,Title,Channel,Upload date,Stock,Summary,Company,Symbol
0,YEoJq_PcOgc,What can reignite Apple shares?,CNBC Television,2024-02-28 17:55:38+00:00,1,i don't expect anything to happen at the share...,Apple,AAPL
1,BOm0zNiaNjg,Alphabet's AI problems: Stock falls 4%,CNBC Television,2024-02-26 19:01:28+00:00,3,The stock of alphabet is down 4%. The stock ne...,Alphabet,GOOGL
2,cTncTPylZQ8,Apple shares touch a 4-month low,CNBC Television,2024-03-05 17:54:20+00:00,1,i'm not particularly very bullish on the eye w...,Apple,AAPL
3,qAITe2Hn8Hc,"Three-Stock Lunch: Broadcom, Kroger & Costco",CNBC Television,2024-03-07 20:47:58+00:00,217,Kroger is up 23% over the past month. This has...,Kroger,KR
4,qAITe2Hn8Hc,"Three-Stock Lunch: Broadcom, Kroger & Costco",CNBC Television,2024-03-07 20:47:58+00:00,23,"finally, costco reporting results after the be...",Costco,COST
...,...,...,...,...,...,...,...,...
213,xbU_b4Pwank,"Target foresees sales rebound, plans new store...",Reuters,2024-03-06 06:46:05+00:00,107,target is aiming for a better year in 2024. on...,Target,TGT
214,q1DoDWjQkDk,Tesla steps up EV price war in China | REUTERS,Reuters,2024-03-01 12:34:11+00:00,11,tesla is stepping up a price war over electric...,Tesla,TSLA
215,gUwKeUkdBVw,Meta resolves issue after thousands report out...,NBC News,2024-03-05 21:30:02+00:00,5,thousands of meta users were not able to sign ...,Meta,META
216,xOqh-4THhF8,Texas Gov. Greg Abbott on border crisis and Tr...,ABC News,2024-03-06 03:30:01+00:00,37,,Abbott,ABT


Step 5: Save

In [ ]:
pd.DataFrame(summarized_video_list).to_csv('youtube_preprocessed_2024.csv')